In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as LSCm
from scipy.interpolate import griddata
from scipy.signal import savgol_filter
from scipy import integrate


In [2]:
from NesrHydrusAnalyst import *

In [3]:
src = '../Datasets/sample3d'

# Modifying data grabbing

In [4]:
# tests get_grid_values
data_frame= read_hydrus_data(folder=src, save_to_csv=False)

In [5]:
v=0
X, Z, M, x_vals, z_vals = get_grid_values(data_frame,variable=v)

In [6]:
# print('x_vals{}, z_vals{}, X{}, Z{}, M{}'.format(x_vals.shape, 
#                                                  z_vals.shape, X.shape, 
#                                                  Z.shape, M.shape))

In [7]:
get_available_timesteps(data_frame)

[0, 15, 60, 120, 180, 360, 720, 1440]

In [8]:
get_full_dimensions(data_frame)

{'x': (0.0, 70.0), 'y': (0.0, 100.0), 'z': (-125.0, 0.0)}

In [9]:
get_legend_range(M.min(), M.max())

array([0.230, 0.240, 0.250, 0.260, 0.270, 0.280, 0.290, 0.300, 0.310,
       0.320, 0.330, 0.340, 0.350])

In [10]:
get_one_line_df(src, simulation_name="Sand Ditch simulation", dims='3d').T

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,WatBalT720,WatBalT1440,hMean0,hMean15,hMean60,hMean120,hMean180,hMean360,hMean720,hMean1440
Sand Ditch simulation,209.75,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,65.245,124.33,-10.221,-10.229,-10.262,-10.306,-10.344,-11.299,-13.464,-16.12


In [12]:
data_frame.head()

,n,x,y,z,Th_T=0,Th_T=15,Th_T=60,Th_T=120,Th_T=180,Th_T=360,Th_T=720,Th_T=1440,H_T=0,H_T=15,H_T=60,H_T=120,H_T=180,H_T=360,H_T=720,H_T=1440
1,1,2.91667,0.0,-2.91667,0.210063,0.191791,0.158732,0.139777,0.130033,0.113877,0.100160,0.089611,-10.2125,-11.2717,-13.5745,-15.3043,-16.5198,-18.8943,-21.5512,-24.7883
2,2,2.91667,0.0,-5.83333,0.210063,0.203564,0.172109,0.152735,0.140227,0.122310,0.107456,0.094662,-10.2125,-10.5847,-12.4606,-14.0738,-15.2490,-17.4841,-20.1368,-23.2386
3,3,70.00000,0.0,-125.00000,0.210063,0.210063,0.210063,0.210063,0.210067,0.203415,0.199713,0.167566,-10.2125,-10.2125,-10.2125,-10.2125,-10.2123,-10.6090,-10.8127,-12.8261
4,4,0.00000,0.0,-125.00000,0.210063,0.210063,0.210063,0.210063,0.210071,0.194964,0.198825,0.167340,-10.2125,-10.2125,-10.2125,-10.2125,-10.2120,-11.0986,-10.8698,-12.8375
5,5,70.00000,100.0,-125.00000,0.210063,0.210063,0.210063,0.210063,0.210067,0.194068,0.199096,0.167400,-10.2125,-10.2125,-10.2125,-10.2125,-10.2122,-11.1508,-10.8534,-12.8323


# Get data from Sand Ditch 3D simulation

In [13]:
src = '../Datasets/H3D2_SandDitch0011'

In [14]:
df= read_hydrus_data(folder=src, save_to_csv=False)

In [16]:
df.head()

,n,x,y,z,Th_T=0,Th_T=5,Th_T=15,Th_T=30,Th_T=60,Th_T=120,...,H_T=5,H_T=15,H_T=30,H_T=60,H_T=120,H_T=180,H_T=240,H_T=500,H_T=720,H_T=1440
1,1,0.0000,0.0,0.00000,0.15,0.056079,0.056170,0.055658,0.054891,0.382578,...,-57.6896,-57.40750,-59.03960,-61.730800,-4.28025,-4.77654,-5.51407,-7.66429,-8.79789,-10.01450
2,2,49.9601,0.0,1.99787,0.15,0.053222,0.053808,0.053949,0.053876,0.266495,...,-68.9306,-66.15350,-65.53300,-65.850000,-8.03020,-6.91749,-7.33861,-9.28291,-9.97427,-9.55989
3,3,49.8802,0.0,3.99627,0.15,0.203979,0.208893,0.211409,0.213525,0.359747,...,-143.6060,-129.86400,-123.52800,-118.526000,-10.08760,-8.96240,-9.38220,-11.33150,-12.00630,-11.55600
4,4,27.3981,0.0,3.09723,0.15,0.245272,0.261515,0.270286,0.390000,0.381931,...,-67.8090,-52.53410,-45.99290,4.978220,-4.42541,-5.89492,-6.76845,-8.86218,-9.78174,-10.86720
5,5,26.6789,0.0,21.08290,0.10,0.072666,0.394500,0.417685,0.430000,0.202791,...,-33.3201,-3.84076,-2.82451,-0.922182,-10.76650,-12.89320,-14.40420,-18.55390,-20.73460,-24.69590


In [18]:
v = 0
X, Z, M, x_vals, z_vals = get_grid_values(df, variable=v)

display(get_available_timesteps(df),
        get_full_dimensions(data_frame),
        get_legend_range(M.min(), M.max()),
        'x_vals{}, z_vals{}, X{}, Z{}, M{}'.format(x_vals.shape, z_vals.shape,
                                                   X.shape, Z.shape, M.shape))

IndexError: list index out of range

In [ ]:
def read_hydrus_data(folder='Current', save_to_csv=True):
    '''
    A function to read both Theat and H files from HYDRUS outputs, 
        then to:
            1- return one dataframe contains both data in a decent format.
            2- save this output to a CSV file (optional, True by default)
    Input:
        The name of the main folder (leave balank for the current folder)
        The option to save_to csv, default =True (Boolean)
    '''
    # Specify the source folder
    if folder=='Current':
        read_dir = os.getcwd()
    else:
        read_dir = folder
        
    # Finding number of nodes in the file
    mesh_file = os.path.join(read_dir, 'MESHTRIA.TXT')
    num_cells = np.array(linecache.getline(mesh_file, 6).split(),int)[0]
    # Define dataframe titles
    titles = ['n', 'x', 'y', 'z'] 
    # Define a list of coordinates
    full_data = [[0,0,0,0]]
    # Set a loop to geather all coordinates from MESHTRIA.TXT file
    for i in range(8, num_cells + 8):
        full_data.append(np.array(linecache.getline(mesh_file, i).split(),float))
    # Convert the list to numpy array then to a dataframe
    coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)
    # Print head and tail of the dataframe to ensure correctness
    # pd.concat([coordinates_df.head(),coordinates_df.tail()])
    
    
    # -----------------------------#
    # To get data from all files   #
    # -----------------------------#
    def get_data_from_file(filename='TH.TXT', caption = 'Theta'):
        '''
        Function to combine all values of a property to a single dataframe 
        inputs:
        filename, the name of the file
        caption, the leading caption of the columns (we will add the portion '_T= xxx')
        where xxx is the timestep
        '''
        # compute number of lines for each timestep
        num_lines = int(math.ceil(num_cells /10.))
        time_steps_remaining = True  # Flag to see if the loop should continue or not.
        times_df = pd.DataFrame([])  # Empty dataframe
        time_loc_start = 2  # The starting cell of the timestep
        while time_steps_remaining:
            line_t = linecache.getline(filename, time_loc_start).split()
            # Check if it is the start of the timestep, otherwise exit
            if line_t[0] == 'Time':
                t = int(line_t[2])
                # Finding the last line of the timestep
                tim_loc_end = num_lines + time_loc_start + 2
                # The starting time is always 0 because steps starts in 1 in HYDRUS
                time_data = [0]  
                # Create the timestep as one long list
                for i in range(time_loc_start + 2, tim_loc_end):
                    time_data.extend(linecache.getline(filename, i).split())
                # Convert the list to DataFrame
                dft=pd.DataFrame(np.array(time_data,float),columns=['{}_T={}'.
                                                                    format(caption,t)])
                if len(times_df) == 0:  # If it is the first timestep
                    times_df = dft
                else:  # Otherwise (for all other timesteps)
                    times_df = pd.concat([times_df, dft], axis=1)
                # Change the start to the probable next timestem (if exist)
                time_loc_start = tim_loc_end + 1
                time_steps_remaining = True if len(linecache.
                                                   getline(filename, 
                                                           time_loc_start)) > 0 else False
                # End IF
        return times_df
    
    # Set the basic dataframe to the coordinates dataframe, to append to it.
    full_df = coordinates_df
    # Looping through the basic output files then to concatenate them all
    for prop in [('TH.TXT','Th'), ('H.TXT','H')]:#, ('V.TXT', 'V')]:
        file_path = os.path.join(read_dir, prop[0])
        # Check if the file exists
        if os.path.isfile(file_path):
            prop_df = get_data_from_file(file_path, prop[1])
            full_df = pd.concat([full_df, prop_df], axis=1)
        else: 
            print ('Warning, the file {} does not exist in the given path'.
                   format(prop[0]))

    # Convert the num column to integer
    full_df[['n']] = full_df[['n']].astype(np.int64)
    # dropping the first row (the zeros row) as it is not necessary
    full_df.drop(0, inplace=True)
    # Saving the resultant dataframe to disk.
    if save_to_csv:
        full_df.to_csv(os.path.join(read_dir, 'nesr_data2.csv'))        
    return full_df

